In [1]:
import os
os.chdir("..")
print(os.getcwd())

import warnings
warnings.filterwarnings("ignore")


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import torch
from torch import nn
from torch import optim
from torch import autograd as ag

from torch.utils.data import DataLoader, Dataset

import segmentation_models_pytorch as smp

%matplotlib inline

import time

In [3]:
import imgaug as ia
import imgaug.augmenters as iaa

In [4]:
# paths to data
TRAIN = os.path.join("data", "train")
TRAIN_MSK = os.path.join("data", "train_mask")
VAL= os.path.join("data", "valid")
VAL_MSK = os.path.join("data", "valid_mask")

In [5]:
class SegmentationData(Dataset):
    """
    Dataset class
    
    :img_path: path to images
    :mask_path: path to masks
    
    """
    
    def __init__(self, img_path, mask_path, size=(224, 224), preprocessing=None, augmentation=None):
        ids = [x.split(".")[0] for x in os.listdir(img_path)]
        self.img_paths = [os.path.join(img_path, x+".jpg") for x in ids]
        self.mask_paths = [os.path.join(mask_path, x+".png") for x in ids]
        self.size = size
        
        
        self.preprocessing = preprocessing
        self.augmentation = augmentation
        
    def __getitem__(self, i):
        # load data
        img = plt.imread(self.img_paths[i])
        img = cv2.resize(img, self.size)
        
        mask = plt.imread(self.mask_paths[i])
        mask = cv2.resize(mask, self.size)
        
        img = img.astype(np.float32)/255
        mask = mask[np.newaxis, :, :, ]
        mask = mask.astype(np.uint8)
        
        # apply preprocessing if needed
        if self.preprocessing:
            img, mask = self.preprocessing(image=img, mask=mask)
    
        # apply augmentation if needed
        if self.augmentation:
            img, mask = self.augmentation(image=img, segmentation_maps=mask)
          
        img = np.rollaxis(img, 2, 0)
        
        return (torch.from_numpy(img.copy()), torch.from_numpy(mask.astype(np.float32)))
            
    def __len__(self):
        return len(self.img_paths)

In [6]:
aug = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.05),
    
    iaa.Sometimes(
        0.5, 
        iaa.Crop(px=(0, 20))
    ),
    
    iaa.SomeOf(
        (0,3),
        [
            iaa.GaussianBlur(sigma=(0, 2)),
             iaa.GammaContrast(per_channel=True, gamma=(0.25,1.75)),
             iaa.LinearContrast(alpha=(0.25,1.75), per_channel=True)
        ]
    ),
    
    iaa.Sometimes(
        0.8,
        [iaa.Sometimes(
                0.3,
                iaa.OneOf([
                    iaa.ElasticTransformation(alpha=20, sigma=1),
                    iaa.ElasticTransformation(alpha=200, sigma=20)
                ])
        ),


        iaa.Sometimes(0.3, 
            iaa.Affine(
                scale={"x": (0.9, 1.1), "y": (0.8, 1.2)},
                rotate=(-20, 20),
                order=[0, 1]
            )
        )]
    )
])

In [28]:
train_data = SegmentationData(TRAIN, TRAIN_MSK)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [29]:
BASE_MODEL = "resnet18"
MODEL_WEGHTS = "imagenet"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [30]:
criterion = smp.utils.losses.BCEDiceLoss()
metrics = [
    smp.utils.metrics.IoUMetric(),
    smp.utils.metrics.FscoreMetric()
]

In [31]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)

In [32]:
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [33]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [34]:
augmentations = {}

In [35]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_noaug.pth')
        augmentations["no_aug"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7


Epoch: 0
valid: 100%|█████████████████| 100/100 [00:03<00:00, 29.72it/s, bce_dice_loss - 0.3935, iou - 0.8271, f-score - 0.8998]
Model saved!

Epoch: 1
valid: 100%|█████████████████| 100/100 [00:03<00:00, 32.99it/s, bce_dice_loss - 0.2872, iou - 0.8719, f-score - 0.9276]
Model saved!

Epoch: 2
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.29it/s, bce_dice_loss - 0.2541, iou - 0.8869, f-score - 0.9364]
Model saved!

Epoch: 3
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.05it/s, bce_dice_loss - 0.2368, iou - 0.8934, f-score - 0.9405]
Model saved!

Epoch: 4
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.24it/s, bce_dice_loss - 0.2298, iou - 0.8995, f-score - 0.9438]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.52it/s, bce_dice_loss - 0.2064, iou - 0.9066, f-score - 0.9482]
Model saved!

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:03<00:00, 27.10it/s, bce_dice_loss - 0.2019, iou - 0.9108, f-score - 0.9504]
Model saved!

In [36]:
aug = iaa.Sequential([
    iaa.Fliplr(0.5),
])

In [37]:
train_data = SegmentationData(TRAIN, TRAIN_MSK, augmentation=aug)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [38]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [39]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [40]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_fliplr.pth')
        augmentations["Flip-lr"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7
        



Epoch: 0
valid: 100%|██████████████████| 100/100 [00:03<00:00, 29.16it/s, bce_dice_loss - 0.3936, iou - 0.8357, f-score - 0.905]
Model saved!

Epoch: 1
valid: 100%|██████████████████| 100/100 [00:03<00:00, 29.16it/s, bce_dice_loss - 0.2779, iou - 0.876, f-score - 0.9297]
Model saved!

Epoch: 2
valid: 100%|██████████████████| 100/100 [00:03<00:00, 26.91it/s, bce_dice_loss - 0.2262, iou - 0.895, f-score - 0.9416]
Model saved!

Epoch: 3
valid: 100%|███████████████████| 100/100 [00:03<00:00, 26.62it/s, bce_dice_loss - 0.267, iou - 0.8733, f-score - 0.927]

Epoch: 4
valid: 100%|█████████████████| 100/100 [00:03<00:00, 27.76it/s, bce_dice_loss - 0.2039, iou - 0.9054, f-score - 0.9478]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:03<00:00, 28.83it/s, bce_dice_loss - 0.2021, iou - 0.9064, f-score - 0.9483]
Model saved!

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:03<00:00, 27.03it/s, bce_dice_loss - 0.1836, iou - 0.9152, f-score - 0.9535]
Model saved!

Epoch: 7
va

In [41]:
aug = iaa.Sequential([
    iaa.SomeOf(
        (0,3),
        [
             iaa.GaussianBlur(sigma=(0, 2)),
             iaa.GammaContrast(per_channel=True, gamma=(0.25,1.75)),
             iaa.LinearContrast(alpha=(0.25,1.75), per_channel=True)
        ]
    ),
])

In [42]:
train_data = SegmentationData(TRAIN, TRAIN_MSK, augmentation=aug)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [43]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [44]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [45]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_contrast.pth')
        augmentations["contrast"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7
        



Epoch: 0
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.07it/s, bce_dice_loss - 0.3757, iou - 0.8344, f-score - 0.9047]
Model saved!

Epoch: 1
valid: 100%|██████████████████| 100/100 [00:02<00:00, 35.28it/s, bce_dice_loss - 0.295, iou - 0.8683, f-score - 0.9248]
Model saved!

Epoch: 2
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.43it/s, bce_dice_loss - 0.2438, iou - 0.8891, f-score - 0.9375]
Model saved!

Epoch: 3
valid: 100%|██████████████████| 100/100 [00:02<00:00, 35.43it/s, bce_dice_loss - 0.2364, iou - 0.8955, f-score - 0.942]
Model saved!

Epoch: 4
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.39it/s, bce_dice_loss - 0.2087, iou - 0.9041, f-score - 0.9473]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.97it/s, bce_dice_loss - 0.1933, iou - 0.9125, f-score - 0.9524]
Model saved!

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.18it/s, bce_dice_loss - 0.1935, iou - 0.9096, f-score - 0.9501]

Epoch: 7
va

In [46]:
aug = iaa.Sequential([
    iaa.Sometimes(
                0.5,
                iaa.OneOf([
                    iaa.ElasticTransformation(alpha=20, sigma=1),
                    iaa.ElasticTransformation(alpha=200, sigma=20)
                ])
        ),
])

In [47]:
train_data = SegmentationData(TRAIN, TRAIN_MSK, augmentation=aug)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [48]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [49]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [50]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_elastic.pth')
        augmentations["elastic"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7
        



Epoch: 0
valid: 100%|█████████████████| 100/100 [00:02<00:00, 33.86it/s, bce_dice_loss - 0.4017, iou - 0.8223, f-score - 0.8965]
Model saved!

Epoch: 1
valid: 100%|█████████████████| 100/100 [00:03<00:00, 33.32it/s, bce_dice_loss - 0.3068, iou - 0.8611, f-score - 0.9213]
Model saved!

Epoch: 2
valid: 100%|█████████████████| 100/100 [00:02<00:00, 33.51it/s, bce_dice_loss - 0.2708, iou - 0.8758, f-score - 0.9304]
Model saved!

Epoch: 3
valid: 100%|█████████████████| 100/100 [00:03<00:00, 33.11it/s, bce_dice_loss - 0.2428, iou - 0.8898, f-score - 0.9391]
Model saved!

Epoch: 4
valid: 100%|█████████████████| 100/100 [00:03<00:00, 33.26it/s, bce_dice_loss - 0.2248, iou - 0.8961, f-score - 0.9426]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.44it/s, bce_dice_loss - 0.2118, iou - 0.9018, f-score - 0.9457]
Model saved!

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.30it/s, bce_dice_loss - 0.2014, iou - 0.9048, f-score - 0.9472]
Model saved!

In [67]:
aug = iaa.Sequential([
    iaa.Sometimes(0.5, 
            iaa.Affine(
                scale={"x": (0.9, 1.1), "y": (0.8, 1.2)},
                rotate=(-20, 20),
                order=[0, 1]
            )
    )
])

In [68]:
train_data = SegmentationData(TRAIN, TRAIN_MSK, augmentation=aug)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [69]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [70]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [71]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_affine.pth')
        augmentations["affine"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7
        



Epoch: 0
valid: 100%|█████████████████| 100/100 [00:03<00:00, 32.43it/s, bce_dice_loss - 0.3665, iou - 0.8343, f-score - 0.9041]
Model saved!

Epoch: 1
valid: 100%|██████████████████| 100/100 [00:02<00:00, 33.94it/s, bce_dice_loss - 0.2675, iou - 0.8781, f-score - 0.932]
Model saved!

Epoch: 2
valid: 100%|█████████████████| 100/100 [00:02<00:00, 33.85it/s, bce_dice_loss - 0.2332, iou - 0.8929, f-score - 0.9405]
Model saved!

Epoch: 3
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.24it/s, bce_dice_loss - 0.2124, iou - 0.9014, f-score - 0.9458]
Model saved!

Epoch: 4
valid: 100%|█████████████████| 100/100 [00:02<00:00, 33.86it/s, bce_dice_loss - 0.1961, iou - 0.9075, f-score - 0.9487]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.18it/s, bce_dice_loss - 0.2013, iou - 0.9055, f-score - 0.9478]

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.34it/s, bce_dice_loss - 0.1805, iou - 0.9166, f-score - 0.9546]
Model saved!

Epoch: 7
va

In [72]:
aug = iaa.Sequential([
    iaa.Sometimes(0.5, 
                  iaa.OneOf([
                      iaa.CoarseSaltAndPepper(p=0.1, size_px=(5,20)),
                      iaa.SaltAndPepper(p=0.01)
                  ])
    )
])

In [73]:
train_data = SegmentationData(TRAIN, TRAIN_MSK, augmentation=aug)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [74]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [75]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [76]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_noise.pth')
        augmentations["noise"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7
        



Epoch: 0
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.19it/s, bce_dice_loss - 0.7329, iou - 0.7064, f-score - 0.8155]
Model saved!

Epoch: 1
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.72it/s, bce_dice_loss - 0.6518, iou - 0.7368, f-score - 0.8366]
Model saved!

Epoch: 2
valid: 100%|██████████████████| 100/100 [00:02<00:00, 35.42it/s, bce_dice_loss - 0.6027, iou - 0.7468, f-score - 0.844]
Model saved!

Epoch: 3
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.92it/s, bce_dice_loss - 0.5986, iou - 0.7546, f-score - 0.8484]
Model saved!

Epoch: 4
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.96it/s, bce_dice_loss - 0.5736, iou - 0.7597, f-score - 0.8523]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.95it/s, bce_dice_loss - 0.5751, iou - 0.7384, f-score - 0.8373]

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:03<00:00, 33.28it/s, bce_dice_loss - 0.5286, iou - 0.7668, f-score - 0.8569]
Model saved!

Epoch: 7
va

In [77]:
aug = iaa.Sequential([
    iaa.Sometimes(0.5, 
            iaa.GaussianBlur(sigma=(0.5, 5))
    )
])

In [78]:
train_data = SegmentationData(TRAIN, TRAIN_MSK, augmentation=aug)
val_data = SegmentationData(VAL, VAL_MSK)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

In [79]:
model = smp.Unet(BASE_MODEL, MODEL_WEGHTS)
optimizer = optim.Adam([
    {"params": model.encoder.parameters(), "lr" : 1e-6},
    {"params": model.decoder.parameters(), "lr" : 1e-4},
])

In [80]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

val_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [81]:
max_score = 0

for i in range(0, 50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = val_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou']:
        max_score = valid_logs['iou']
        torch.save(model, 'models/best_model_unet_resnet18_blur.pth')
        augmentations["blur"] = {"iou": valid_logs["iou"], 
                           "f-score": valid_logs["f-score"] 
                          }
        print('Model saved!')
            
    if i == 25:
        optimizer.param_groups[1]['lr'] = 1e-5
        optimizer.param_groups[0]['lr'] = 1e-7
        



Epoch: 0
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.97it/s, bce_dice_loss - 0.3929, iou - 0.8286, f-score - 0.9008]
Model saved!

Epoch: 1
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.94it/s, bce_dice_loss - 0.3118, iou - 0.8565, f-score - 0.9173]
Model saved!

Epoch: 2
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.31it/s, bce_dice_loss - 0.2554, iou - 0.8817, f-score - 0.9334]
Model saved!

Epoch: 3
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.18it/s, bce_dice_loss - 0.2286, iou - 0.8936, f-score - 0.9407]
Model saved!

Epoch: 4
valid: 100%|██████████████████| 100/100 [00:02<00:00, 35.07it/s, bce_dice_loss - 0.219, iou - 0.8999, f-score - 0.9444]
Model saved!

Epoch: 5
valid: 100%|█████████████████| 100/100 [00:02<00:00, 34.96it/s, bce_dice_loss - 0.2012, iou - 0.9077, f-score - 0.9491]
Model saved!

Epoch: 6
valid: 100%|█████████████████| 100/100 [00:02<00:00, 35.04it/s, bce_dice_loss - 0.1983, iou - 0.9096, f-score - 0.9503]
Model saved!

In [84]:
import json

In [83]:
import pandas as pd

In [85]:
with open("results/augmentations.json", "w") as f:
    json.dump(augmentations, f)

In [90]:
pd.read_json("results/augmentations.json").T

,f-score,iou
Flip-lr,0.964886,0.935092
affine,0.967146,0.939448
blur,0.961013,0.928530
contrast,0.967045,0.938485
elastic,0.965203,0.935752
no_aug,0.959862,0.927140
noise,0.908004,0.839923


In [ ]:
pd.read_json()

In [82]:
augmentations

{'Flip-lr': {'f-score': 0.9648858624696733, 'iou': 0.9350917828083036},
 'affine': {'f-score': 0.9671458303928373, 'iou': 0.939448472261429},
 'blur': {'f-score': 0.9610132664442063, 'iou': 0.9285300475358964},
 'contrast': {'f-score': 0.9670450282096859, 'iou': 0.9384848111867903},
 'elastic': {'f-score': 0.9652028703689575, 'iou': 0.9357521748542788},
 'no_aug': {'f-score': 0.959861744046211, 'iou': 0.9271400225162507},
 'noise': {'f-score': 0.9080041667819023, 'iou': 0.8399234387278557}}